In [42]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates

import ind.alpha_base10 as base10


%load_ext autoreload
%autoreload 2
%aimport ind.alpha_base10



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# #读取已经缓存的复权数据，并预处理
stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)
## 日无风险回报
ret_fs_data = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs_data = (ret_fs_data['high'].astype(np.float32)+ret_fs_data['low'].astype(np.float32))/2 * 0.01
ret_fs_daily = ret_fs_data/252

ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)


#############测试#######################
# ['000001','000008','600600','000729']
# stock_df = stock_df_full.loc[(slice(None),stock_df_full.index.get_level_values(1).unique().to_list()[0:22]),:]
# ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)

stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)


# def generate_alpha_factors(codes, data_df=None, ret_fs=None, ret=None, ret_excess=None, market_value=None, ret_excess_market=None):
#     import ind.alpha_base10 as base10
#     from base.JuUnits import excute_for_multidates
#     import numpy as np
#     import pandas as pd

#     df = data_df.loc[(slice(None),codes),:]
#     ret_ = ret.loc[(slice(None),codes)]
#     ret_excess_ = ret_excess.loc[(slice(None),codes)]
    

#     alpha_beta = base10.camp_beta_alpha(ret_excess_,ret_excess_market)
#     mom = base10.momentum(ret_,ret_fs)
#     size_lg = base10.sizelg(df)
#     bp = base10.bp(df)
    
#     return pd.concat([alpha_beta, mom, size_lg, bp], axis=1).loc[(slice(None),codes),:]


# worker=6
# task = task_chunk_split(stock_codes, worker)
# results = parallal_task(worker, generate_alpha_factors, task, 
#                         data_df=stock_df, ret_fs=ret_fs_daily, ret=ret_t, 
#                         ret_excess=ret_t_excess, market_value=market_value_t,
#                         ret_excess_market=ret_excess_market_t)
# results = pd.concat(results).sort_index()

####使用全局数据的项目，注意不要分步计算
earnings_yield = base10.earnings_yield(ret_t,market_value_t,stock_df['industry']) 
print('earnings_yield')
liquidity = base10.liquidity(stock_df)
print('liquidity')
resvol = base10.resvol(ret_t, ret_fs_daily, ret_t_excess, results['sizelg'], results['beta'], results['residual'])
print('resvol')
sizenl = base10.sizenl(results['sizelg'])

final = pd.concat([results, earnings_yield, liquidity, resvol, sizenl],axis=1).sort_index()
save_cache('base10_train', final, cache_type=CACHE_TYPE.FACTOR)

earnings_yield
liquidity
resvol


In [45]:
# load_cache('base10_train',cache_type=CACHE_TYPE.FACTOR)


beta     alpha  residual  momentum     sizelg  \
date       code                                                        
2021-12-31 000026  1.143621 -0.001795  0.010491 -0.000362  22.313660   
           000027  1.032228 -0.001094 -0.002058  0.000339  24.165215   
           000028  0.386453 -0.000264  0.042254 -0.000486  23.464885   
           000030  1.189952  0.000042  0.008494 -0.001542  23.268340   
           000031  0.706616 -0.000466  0.009315 -0.000593  23.495101   

                         bp  earnings_yield  liquidity    resvol    sizenl  
date       code                                                             
2021-12-31 000026  1.628039        0.041158   0.126414 -0.023644 -0.413896  
           000027  1.373038        0.079392   0.447063 -0.020278 -0.495062  
           000028  1.039340        0.057565  -0.465150 -0.010124 -1.013241  
           000030  1.705557        0.024221  -0.138011  0.026394 -1.037021  
           000031  0.848692       -0.071913  -0.597536 -0.000199 -1.004902

In [61]:
final.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3247889 entries, (Timestamp('2017-01-03 00:00:00'), '000001') to (Timestamp('2021-12-31 00:00:00'), '605599')
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   beta            float64
 1   alpha           float64
 2   residual        float64
 3   momentum        float64
 4   sizelg          float64
 5   bp              float64
 6   earnings_yield  float64
 7   liquidity       float64
 8   resvol          float64
 9   sizenl          float64
dtypes: float64(10)
memory usage: 389.3+ MB
